# 🌵 Qwen3 Tool Calling Training with Unsloth

This notebook trains Qwen 3 models for tool calling using Unsloth, a fast and memory-efficient LoRA fine-tuning library.

**Features:**
- 2x faster training than standard HuggingFace Trainer
- 80% less memory with 4-bit quantization
- Sparse checkout: downloads only needed folders (~10MB)
- Uses Unsloth default hyperparameters

**Requirements:**
- GPU runtime (T4 or better recommended)
- ~6GB VRAM for Qwen3-0.6B
- ~15-30 minutes training time

---

## 📋 Step 1: Check GPU & Install Unsloth

First verify we have a GPU, then install Unsloth using the official installation method.

In [ ]:
%%capture

# Check GPU
!nvidia-smi

print("\n" + "="*60)
print("Installing Unsloth (this may take 2-3 minutes)...")
print("="*60 + "\n")

import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

print("\n✅ Unsloth installed successfully!")

## 📥 Step 2: Clone Repository (Sparse Checkout)

Use sparse checkout to download only the folders we need:
- `pebble_qwen_unsloth/` - Training code
- `pebble_qwen/data/` - Dataset and tools

In [ ]:
import os

# Clean up any existing clone
if os.path.exists('cactus'):
    !rm -rf cactus

print("Cloning repository with sparse checkout...\n")

# Clone with sparse checkout
!git clone \
    --depth 1 \
    --filter=blob:none \
    --sparse \
    --branch noah/pebble-qwen-unsloth \
    https://github.com/BruinAI/cactus.git

# Change to repo directory
%cd cactus

# Set sparse checkout to only include needed folders
!git sparse-checkout set pebble_qwen_unsloth pebble_qwen/data

print("\n" + "="*60)
print("Repository Structure:")
print("="*60)
!ls -R pebble_qwen_unsloth
!ls -R pebble_qwen/data

print("\n✅ Repository cloned successfully!")

## ⚙️ Step 3: Configuration & Setup

Set hyperparameters and configuration. **Modify these values as needed!**

In [ ]:
# ============================================================================
# CONFIGURATION - Modify these values as needed!
# ============================================================================

# Model configuration
MODEL_ID = "unsloth/Qwen3-0.6B"  # Options: Qwen3-0.6B, Qwen3-1.7B, Qwen3-8B
MAX_SEQ_LENGTH = 2048
LOAD_IN_4BIT = True

# Training hyperparameters (Unsloth defaults)
LEARNING_RATE = 2e-4
NUM_EPOCHS = 1
BATCH_SIZE = 32
GRADIENT_ACCUMULATION_STEPS = 1  # Effective batch size: 8

# LoRA hyperparameters (Unsloth defaults)
LORA_RANK = 16
LORA_ALPHA = 16
LORA_DROPOUT = 0

# Data configuration
TRAIN_TEST_SPLIT = 0.1  # 10% validation split (set to 0 to disable)
ENABLE_EVAL = True  # Set to False to skip evaluation and save VRAM

# Output configuration
OUTPUT_DIR = "outputs"
FINAL_MODEL_DIR = "qwen3_tool_calling_trained"

# Optional: Mount Google Drive for persistent storage
USE_GOOGLE_DRIVE = False  # Set to True to save models to Drive

print("="*60)
print("Configuration:")
print("="*60)
print(f"Model: {MODEL_ID}")
print(f"Max sequence length: {MAX_SEQ_LENGTH}")
print(f"Quantization: {'4-bit' if LOAD_IN_4BIT else '16-bit (full precision)'}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Epochs: {NUM_EPOCHS}")
print(f"Batch size: {BATCH_SIZE} (effective: {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS})")
print(f"LoRA rank: {LORA_RANK}, alpha: {LORA_ALPHA}")
print("="*60)

In [ ]:
# Optional: Mount Google Drive
if USE_GOOGLE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    FINAL_MODEL_DIR = f"/content/drive/MyDrive/{FINAL_MODEL_DIR}"
    print(f"✅ Google Drive mounted! Models will be saved to: {FINAL_MODEL_DIR}")
else:
    print("ℹ️ Not using Google Drive. Models will be saved to local Colab storage.")

## 🤖 Step 4: Load Model & Tokenizer

Load the model with the configured quantization setting (see Configuration above).

In [ ]:
import torch
from unsloth import FastLanguageModel

print("Loading model and tokenizer...\n")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_ID,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,  # Auto-detect
    load_in_4bit=LOAD_IN_4BIT,  # Use configured quantization setting
)

print(f"✅ Model loaded: {MODEL_ID}")
print(f"✅ Quantization: {'4-bit' if LOAD_IN_4BIT else '16-bit (full precision)'}")
print(f"✅ Tokenizer loaded with vocab size: {len(tokenizer)}")
print(f"✅ Device: {next(model.parameters()).device}")

## 🎯 Step 5: Apply LoRA

Apply Low-Rank Adaptation for efficient fine-tuning.

In [ ]:
print("Applying LoRA...\n")

model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_RANK,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"],
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    use_gradient_checkpointing="unsloth",  # 30% memory savings
    random_state=3407,
    max_seq_length=MAX_SEQ_LENGTH,
)

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"✅ LoRA applied: rank={LORA_RANK}, alpha={LORA_ALPHA}")
print(f"✅ Trainable parameters: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")

## 📊 Step 6: Load & Prepare Dataset

Load the tool calling dataset and format it for training.

In [ ]:
import sys
import json
from datasets import Dataset

# Add pebble_qwen_unsloth to path
sys.path.insert(0, '/content/cactus')

from pebble_qwen_unsloth.data.format_dataset import (
    format_qwen3_dataset,
    load_tools,
    load_dataset
)

print("Loading dataset and tools...\n")

# Load tools and dataset
tools = load_tools('pebble_qwen/data/tools.json')
dataset_samples = load_dataset('pebble_qwen/data/synthetic_finetune_dataset.json')

print(f"✅ Loaded {len(tools)} tools")
print(f"✅ Loaded {len(dataset_samples)} dataset samples")

# Format dataset
print("\nFormatting examples for Qwen 3 tool calling...")
formatted_data = []

for sample in dataset_samples:
    formatted = format_qwen3_dataset(sample, tools, tokenizer)
    if formatted:
        # Combine all role messages into a single text string
        full_text = "".join(msg['text'] for msg in formatted)
        formatted_data.append({'text': full_text})

full_dataset = Dataset.from_list(formatted_data)

# Split dataset
if ENABLE_EVAL and TRAIN_TEST_SPLIT > 0:
    split = full_dataset.train_test_split(test_size=TRAIN_TEST_SPLIT, seed=42)
    train_dataset = split['train']
    eval_dataset = split['test']
else:
    train_dataset = full_dataset
    eval_dataset = None

print(f"\n✅ Formatted {len(formatted_data)} samples")
print(f"✅ Training split: {len(train_dataset)} samples")
if eval_dataset:
    print(f"✅ Validation split: {len(eval_dataset)} samples")

## 👀 Step 7: Preview Training Examples

Let's look at some formatted training examples.

In [ ]:
import random

print("="*60)
print("Sample Training Examples (2 random samples)")
print("="*60)

indices = random.sample(range(len(train_dataset)), min(2, len(train_dataset)))

for i, idx in enumerate(indices, 1):
    example = train_dataset[idx]
    text = example['text']
    
    print(f"\n{'─'*60}")
    print(f"Example {i} (Index: {idx})")
    print(f"{'─'*60}")
    
    # Truncate long text for display
    if len(text) > 800:
        print(text[:800] + "\n...[truncated]")
    else:
        print(text)

print(f"\n{'='*60}\n")

## 🎓 Step 8: Setup Trainer

Configure the SFTTrainer with our hyperparameters.

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

print("Setting up trainer...\n")

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=max(BATCH_SIZE // 4, 1),
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    warmup_steps=10,
    num_train_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    save_strategy="epoch",
    eval_strategy="epoch" if eval_dataset else "no",
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    args=training_args,
    packing=False,
)

total_steps = len(train_dataset) * NUM_EPOCHS // (BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS)

print("✅ Trainer initialized")
print(f"✅ Total training steps: {total_steps}")
print(f"✅ Estimated time: ~{total_steps * 2 // 60}-{total_steps * 3 // 60} minutes")

## 🚀 Step 9: Train!

Start the training process. This will take 15-30 minutes depending on your GPU.

In [ ]:
print("="*60)
print("Starting training")
print("="*60)
print("This may take 15-30 minutes...\n")

# Train!
trainer.train()

print("\n" + "="*60)
print("✅ Training complete!")
print("="*60)

## 🧪 Step 10: Test Trained Model

Test the trained model with some examples.

In [ ]:
print("="*60)
print("Testing Trained Model")
print("="*60)

# Prepare model for inference
FastLanguageModel.for_inference(model)

# Test cases
test_cases = [
    {'input': 'Remember to buy groceries tomorrow',
     'output': {'function_call': {'name': 'create_note', 'arguments': {}}}},
    {'input': 'Set an alarm for 7:30 AM',
     'output': {'function_call': {'name': 'set_alarm', 'arguments': {}}}},
    {'input': 'Set a timer for 10 minutes',
     'output': {'function_call': {'name': 'set_timer', 'arguments': {}}}}
]

for i, sample in enumerate(test_cases, 1):
    # Format the example
    formatted = format_qwen3_dataset(sample, tools, tokenizer)
    if not formatted:
        continue
    
    # Extract prompt (system + user)
    prompt = "".join(msg['text'] for msg in formatted if msg['role'] in ['system', 'user'])
    prompt += "<think>\n\n</think>\n\n"
    
    # Generate
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            temperature=0.0,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    generated = response[len(prompt):]
    
    print(f"\n{'─'*60}")
    print(f"Test {i}: {sample['input']}")
    print(f"{'─'*60}")
    print("Generated:")
    print(generated.strip())

print(f"\n{'='*60}\n")

## 💾 Step 11: Save Model

Save the trained model in HuggingFace safetensors format.

In [ ]:
import os

print("="*60)
print("Saving model in HuggingFace safetensors format")
print("="*60)
print(f"Output directory: {FINAL_MODEL_DIR}\n")

# Save merged model (LoRA merged with base weights)
model.save_pretrained_merged(
    FINAL_MODEL_DIR,
    tokenizer,
    save_method="merged_16bit"
)

print(f"\n✅ Model saved to: {FINAL_MODEL_DIR}")
print("\nSaved files:")
for f in os.listdir(FINAL_MODEL_DIR):
    if os.path.isfile(os.path.join(FINAL_MODEL_DIR, f)):
        size = os.path.getsize(os.path.join(FINAL_MODEL_DIR, f)) / (1024 * 1024)
        print(f"  {f:<30} {size:>10.2f} MB")

print("\n" + "="*60)
print("✅ Training and saving complete!")
print("="*60)

## 📥 Step 13 (Optional): Download Model

Download the trained model to your local machine.

In [ ]:
# Zip the model directory
!zip -r qwen3_trained_model.zip {FINAL_MODEL_DIR}

# Download
from google.colab import files
files.download('qwen3_trained_model.zip')

print("✅ Model zip file will be downloaded to your computer.")

## 📝 Step 13: Usage Instructions

How to load and use your trained model:

In [ ]:
print("""
═══════════════════════════════════════════════════════════════
🎉 Training Complete! Here's how to use your model:
═══════════════════════════════════════════════════════════════

Option 1: Load in Python
─────────────────────────
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("{FINAL_MODEL_DIR}")
tokenizer = AutoTokenizer.from_pretrained("{FINAL_MODEL_DIR}")

Option 2: Continue training with Unsloth
─────────────────────────────────────────
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="{FINAL_MODEL_DIR}",
    max_seq_length=2048,
    load_in_4bit=True,
)

Option 3: Push to HuggingFace Hub
──────────────────────────────────
model.push_to_hub_merged(
    "your-username/qwen3-tool-calling",
    tokenizer,
    save_method="merged_16bit",
    token="hf_..."
)

═══════════════════════════════════════════════════════════════
""")

---

## 🔗 Resources

- **GitHub Repository**: [github.com/cactus-compute/cactus](https://github.com/cactus-compute/cactus)
- **Unsloth Documentation**: [docs.unsloth.ai](https://docs.unsloth.ai/)
- **Unsloth GitHub**: [github.com/unslothai/unsloth](https://github.com/unslothai/unsloth)

---

**Created by**: Noah Cylich  
**License**: Apache 2.0  
**Model**: Qwen3 (Alibaba Cloud)  
**Framework**: Unsloth + HuggingFace Transformers